In [6]:
## Example inspired by Oregon summer school lectures on probabilistic programming
## Code for simulation and visualization inspired from getting started tutorial in pymunk website

# import sys, random
# import pygame
# from pygame.locals import *
import pymunk
# from pymunk.pygame_util import DrawOptions
import pymc3 as pm
# import matplotlib.pyplot as plt
# import arviz as az
import theano
import theano.tensor as tt
import numpy as np

def add_ball(space):
    mass = 1
    radius = 14
    moment = pymunk.moment_for_circle(mass, 0, radius) # 1
    body = pymunk.Body(mass, moment) # 2
    x = random.randint(120, 480)
    body.position = x, 550 # 3
    shape = pymunk.Circle(body, radius) # 4
    space.add(body, shape) # 5
    return shape

def add_static_L(space,x,y,θ):
    body = pymunk.Body(body_type = pymunk.Body.STATIC)
    body.position = (x, y)
    l1 = pymunk.Segment(body, (0, 0), (100, 0+θ), 2)
    space.add(l1)
def add_bucket(space):
    floor = pymunk.Body(body_type = pymunk.Body.STATIC)
    floor.position = (400, 200)

    left_wall = pymunk.Body(body_type = pymunk.Body.STATIC)
    left_wall.position = (400, 200)
    
    right_wall = pymunk.Body(body_type = pymunk.Body.STATIC)
    right_wall.position = (500, 200)
    
    l1 = pymunk.Segment(floor, (0, 0), (100, 0), 2)
    l2 = pymunk.Segment(left_wall, (0, 0), (0, 50), 2)
    l3 = pymunk.Segment(right_wall, (0, 50), (0, 0), 2)

    space.add(l1, l2, l3) # 3
    return l1,l2,l3


@theano.compile.ops.as_op(itypes=[tt.lscalar, tt.lscalar, tt.lscalar,
                                  tt.lscalar, tt.lscalar, tt.lscalar],
                          otypes=[tt.lscalar])
def simulation(xl1, yl1, θl1,
               xl2, yl2, θl2):

    space = pymunk.Space()
    space.gravity = (0.0, -900.0)
    
    add_static_L(space, xl1, yl1, θl1)
    add_static_L(space, xl2, yl2, θl2)
    add_bucket(space)
    ball = add_ball_static(space)
    
    [space.step(1/50.0) for i in range(0,50000)]
    return np.array(ball.body.position.x, dtype=np.int64) if ball.body.position.y > 210 else np.array(0, dtype=np.int64)

In [7]:
with pm.Model() as model:
    xl1 = pm.DiscreteUniform('xl1', lower=0,upper=500)
    yl1 = pm.DiscreteUniform('yl1', lower=150,upper=500)
    θl1 = pm.DiscreteUniform('θl1', lower=-20,upper=20)

    xl2 = pm.DiscreteUniform('xl2', lower=0,upper=500)
    yl2 = pm.DiscreteUniform('yl2', lower=150,upper=500)
    θl2 = pm.DiscreteUniform('θl2', lower=-20,upper=20)

    
        
    obs = pm.Normal('obs',
                    mu=simulation(xl1, yl1, θl1, xl2, yl2, θl2),
                    sigma=.1, observed=484)

    trace = pm.sample(10)


Only 10 samples in chain.


Multiprocess sampling (4 chains in 4 jobs)


CompoundStep


>Metropolis: [θl2]


>Metropolis: [yl2]


...

Sampling 4 chains: 100%|█████████▉| 2036/2040 [11:09<00:04,  1.09s/draws]

Sampling 4 chains: 100%|█████████▉| 2037/2040 [11:10<00:03,  1.08s/draws]

Sampling 4 chains: 100%|█████████▉| 2038/2040 [11:11<00:02,  1.06s/draws]

Sampling 4 chains: 100%|█████████▉| 2039/2040 [11:12<00:01,  1.06s/draws]

Sampling 4 chains: 100%|██████████| 2040/2040 [11:13<00:00,  1.04s/draws]

In [8]:
[t for t in trace]  # Solutions

[{'xl1': 89, 'yl1': 486, 'θl1': -2, 'xl2': 222, 'yl2': 394, 'θl2': -11},
 {'xl1': 89, 'yl1': 486, 'θl1': -2, 'xl2': 222, 'yl2': 394, 'θl2': -11},
 {'xl1': 89, 'yl1': 486, 'θl1': -7, 'xl2': 222, 'yl2': 373, 'θl2': -11},
 {'xl1': 89, 'yl1': 486, 'θl1': -1, 'xl2': 222, 'yl2': 373, 'θl2': -11},
 {'xl1': 89, 'yl1': 494, 'θl1': -1, 'xl2': 222, 'yl2': 348, 'θl2': -11},
 {'xl1': 89, 'yl1': 493, 'θl1': -1, 'xl2': 222, 'yl2': 348, 'θl2': -11},
 {'xl1': 142, 'yl1': 493, 'θl1': -8, 'xl2': 222, 'yl2': 371, 'θl2': -11},
 {'xl1': 142, 'yl1': 493, 'θl1': -8, 'xl2': 222, 'yl2': 379, 'θl2': -11},
 {'xl1': 128, 'yl1': 493, 'θl1': -9, 'xl2': 222, 'yl2': 379, 'θl2': -11},
 {'xl1': 128, 'yl1': 493, 'θl1': -12, 'xl2': 222, 'yl2': 380, 'θl2': -11}]